#### Mục tiêu: Xây dựng mô hình có độ chính xác trên tập validation khoảng 84%

In [1]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding

In [3]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json


--2020-07-11 08:56:29--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 172.217.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.02s   

2020-07-11 08:56:29 (243 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [5]:
with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


import numpy as np
labels = np.array(labels)

In [7]:
datastore[0]

{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
 'is_sarcastic': 0}

In [8]:
sentences[0]

"former versace store clerk sues over secret 'black code' for minority shoppers"

In [41]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [15]:
# Chia features tập train
training_sentences = sentences[0:training_size]
# FIXME: Chia features tập test
testing_sentences = sentences[training_size:]
# FIXME: Chia labels tập train
training_labels_sentences = labels[0:training_size]
# FIXME: Chia labels tập test
testing_labels_sentences = labels[training_size:]

In [24]:
# vocab_size = 10000
# embedding_dim = 16
# max_length = 100
# trunc_type='post'
# padding_type='post'
# oov_tok = "<OOV>"
# training_size = 20000
# FIXME: Tạo tokenizer
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)

# FIXME: Đưa từ vào tokenizer để tạo từ điển

tokenizer.fit_on_texts(training_sentences)

# FIXME: Biến các câu train thành sequences

training_sequences = tokenizer.texts_to_sequences(training_sentences)

# FIXME: Padding cho các train sequences

training_padded = pad_sequences(
                                  training_sequences,
                                 maxlen = 140,
                                 padding = padding_type,
                                 truncating = "post")

# FIXME: Biến các câu test thành sequences và padding tương ứng

test_sequences = tokenizer.texts_to_sequences(testing_sentences)

test_padded =  pad_sequences(
                                  test_sequences,
                                 maxlen = 140,
                                 padding = padding_type,
                                 truncating = "post")


In [30]:
tokenizer.word_index

{'<OOV>': 1,
 'to': 2,
 'of': 3,
 'the': 4,
 'in': 5,
 'for': 6,
 'a': 7,
 'on': 8,
 'and': 9,
 'with': 10,
 'is': 11,
 'new': 12,
 'trump': 13,
 'man': 14,
 'from': 15,
 'at': 16,
 'about': 17,
 'you': 18,
 'by': 19,
 'this': 20,
 'after': 21,
 'be': 22,
 'up': 23,
 'out': 24,
 'that': 25,
 'how': 26,
 'as': 27,
 'it': 28,
 'not': 29,
 'are': 30,
 'your': 31,
 'what': 32,
 'his': 33,
 'all': 34,
 'he': 35,
 'will': 36,
 'who': 37,
 'just': 38,
 'has': 39,
 'more': 40,
 'one': 41,
 'year': 42,
 'into': 43,
 'report': 44,
 'have': 45,
 'why': 46,
 'over': 47,
 'area': 48,
 'u': 49,
 'donald': 50,
 'says': 51,
 'day': 52,
 'can': 53,
 's': 54,
 'first': 55,
 'woman': 56,
 'time': 57,
 'like': 58,
 'get': 59,
 'old': 60,
 'no': 61,
 'her': 62,
 "trump's": 63,
 'off': 64,
 'now': 65,
 'an': 66,
 'obama': 67,
 'life': 68,
 'people': 69,
 'women': 70,
 'house': 71,
 "'": 72,
 'was': 73,
 'still': 74,
 'white': 75,
 'back': 76,
 'make': 77,
 'than': 78,
 'clinton': 79,
 'down': 80,
 'when': 8

In [80]:
len(tokenizer.word_index)

25637

In [36]:

from tensorflow.keras.layers import Bidirectional 
from tensorflow.keras.layers import  LSTM
from keras import regularizers

Using TensorFlow backend.


In [68]:
# Xây dựng model

# FIXME: Tạo model Sequential

model = Sequential()

# FIXME: Thêm một lớp Embedding

model.add(Embedding(vocab_size,64))

# FIXME: Thêm một lớp Bidirectional có LSTM 

model.add(Bidirectional(LSTM(64,return_sequences=True)))

# FIXME: Thêm một lớp Bidirectional có LSTM nữa.
# sử dụng h ở các lớp trước
model.add(Bidirectional(LSTM(32)))

# FIXME : Thêm 1 chút muối (regularizers) để đêm nằm không khóc :3

model.add(Dense(64,activation="relu",kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1,activation="sigmoid"))

In [69]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, None, 128)         66048     
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 64)                41216     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 751,489
Trainable params: 751,489
Non-trainable params: 0
_________________________________________________________________


In [71]:
# FIXME: Compile Optimizer và Loss function

model.compile(loss="binary_crossentropy",optimizer="adam",metrics="acc")

In [72]:
# FIXME: Tiến hành training

history =model.fit(training_padded,training_labels_sentences,
          epochs=15,
          validation_data=(test_padded,testing_labels_sentences))

Epoch 1/15
625/625 [==============================] - 19s 31ms/step - loss: 0.4814 - acc: 0.8117 - val_loss: 0.3396 - val_acc: 0.8612
Epoch 2/15
625/625 [==============================] - 18s 30ms/step - loss: 0.2037 - acc: 0.9241 - val_loss: 0.3857 - val_acc: 0.8487
Epoch 3/15
625/625 [==============================] - 18s 29ms/step - loss: 0.1203 - acc: 0.9617 - val_loss: 0.4211 - val_acc: 0.8489
Epoch 4/15
625/625 [==============================] - 18s 30ms/step - loss: 0.0805 - acc: 0.9766 - val_loss: 0.4857 - val_acc: 0.8398
Epoch 5/15
625/625 [==============================] - 18s 30ms/step - loss: 0.0562 - acc: 0.9857 - val_loss: 0.5511 - val_acc: 0.8456
Epoch 6/15
625/625 [==============================] - 18s 30ms/step - loss: 0.0459 - acc: 0.9879 - val_loss: 0.5678 - val_acc: 0.8365
Epoch 7/15
625/625 [==============================] - 18s 29ms/step - loss: 0.0361 - acc: 0.9912 - val_loss: 0.6135 - val_acc: 0.8375
Epoch 8/15
625/625 [==============================] - 18s 30ms

In [77]:
print("the best :",max(history.history['val_acc']))
print("acc_val:",history.history["val_acc"])

the best : 0.8612312078475952
acc_val: [0.8612312078475952, 0.8487107157707214, 0.8488597273826599, 0.8397674560546875, 0.8455805778503418, 0.8364883065223694, 0.8375316858291626, 0.8387240767478943, 0.8390222191810608, 0.8294827938079834, 0.8290356397628784, 0.8327619433403015, 0.8296318650245667, 0.8363392353057861, 0.8290356397628784]


In [ ]:
# FIXME: Tiến hành dự đoán

In [ ]:
test_sen = ["Everyone seems normal until you get to know them."]

# FIXME: Chuyển câu thành Sequences
test_sen_seq = tokenizer.texts_to_sequences(test) 
# FIXME: padding cho câu này
pad_test_sen = pad_sequences(
                                  test_sequences,
                                 maxlen = 140,
                                 padding = padding_type,
                                 truncating = "post")
# FIXME: Tiến hành dự đoán

model.predict(pad_test_sen)